In [ ]:
import os
import cv2
import numpy as np
from cv2 import VideoWriter, VideoWriter_fourcc

width = 640
height = 480
FPS = 10
seconds = 15  # 1.0
frames = int(FPS * seconds)

# Could use "test.mp4" and the "H264" fourcc codec below.
FILENAME = "test.webm"
if os.path.exists(FILENAME):
    os.remove(FILENAME)

fourcc = VideoWriter_fourcc(*"VP80")
video = VideoWriter(FILENAME, fourcc, float(FPS), (width, height))

left_x = 0  # int(width * -0.5)
right_x = width  # int(width * 1.5)
top_y = 0  # int(height * -0.5)
bottom_y = height  # int(height * 1.5)

resolution = width * 0.04
num_columns = int(width / resolution)  # int((right_x - left_x) / resolution)
num_rows = int(height / resolution)  # int((bottom_y - top_y) / resolution)

grid = np.ones((num_columns, num_rows))
lengths = resolution * 1.0 * np.ones((num_columns, num_rows))

default_angle = np.pi * 0.25
# grid *= default_angle

x_clr = 255.0 / num_columns
y_clr = 255.0 / num_rows

# Create grids for x and y
ys, xs = np.meshgrid(np.arange(num_rows), np.arange(num_columns))

# Compute pixel locations for all xs and ys
xs1 = xs * resolution
ys1 = ys * resolution

grid *= (ys / num_rows) * np.pi

xs1_int = xs1.astype(int)
ys1_int = ys1.astype(int)

# saturate_x = lambda value  int(np.max(np.min(value, width), 0))
# saturate_y = lambda value: int(np.max(np.min(value, height), 0))
saturate = lambda arr, max_value: np.clip(arr, 0, max_value).astype(int)
saturate_x = lambda arr: saturate(arr, width - 1)
saturate_y = lambda arr: saturate(arr, height - 1)

thickness = np.ones((num_columns, num_rows), dtype=np.uint8)

for frame_num in range(frames):
    # Start with a white background.
    frame = np.ones((height, width, 3), dtype=np.uint8) * 255
    # cv2.line(frame, (0, 0), (200 * paint_x, 200), (0, 0, 255), 5)
    cos_grid = np.cos(grid)
    sin_grid = np.sin(grid)

    line_offsets_x = np.multiply(cos_grid, lengths)
    line_offsets_y = np.multiply(sin_grid, lengths)
    xs2 = (xs1 + line_offsets_x).astype(int)
    ys2 = (ys1 + line_offsets_y).astype(int)

    # Draw the grid on this
    for y in range(num_rows):
        for x in range(num_columns):
            color = (x_clr * x, y_clr * y, 255)
            px1 = xs1_int[x][y]
            py1 = ys1_int[x][y]
            px2 = xs2[x][y]
            py2 = ys2[x][y]
            pthickness = thickness[x][y]
            # cv2.line(
            #     frame,
            #     (px1, py1),
            #     (px2, py2),
            #     color,
            #     thickness=pthickness,
            #     lineType=cv2.LINE_AA,
            # )

    # Draw a curve
    num_steps = 50
    points = np.zeros((num_steps, 2), dtype=np.int32)
    # start at:
    # x = 200
    # y = 200
    step_length = 2.0

    #for i in range(num_steps):
    #    points[i][0] = x
    #    points[i][1] = y
    #    column_index = saturate_x(x / resolution)
    #    row_index = saturate_y(y / resolution)
    #    angle = grid[column_index][row_index]
    #    x_step = step_length * np.cos(angle)
    #    y_step = step_length * np.sin(angle)
    #
    #    x = x + x_step
    #    y = y + y_step
    #
    #points = points.reshape((-1, 1, 2))
    #frame = cv2.polylines(frame, [points], False, (0, 0, 0))

    # Try vectorized approach
    v_rows = num_rows
    v_cols = num_columns
    # For each v_row, v_col, we want a list of points
    parr = np.zeros((v_cols, v_rows, num_steps, 2))
    # Current x and y:
    pcursor = np.zeros((v_cols, v_rows, 2))
    for r in range(v_rows):
        for c in range(v_cols):
            # Initial x, y
            pcursor[c][r][0] = c * resolution
            pcursor[c][r][1] = r * resolution

    for i in range(num_steps):
        # Assign cursor x, y to the step index.
        parr[:, :, i] = pcursor[:, :]

        x_index = saturate(pcursor[:, :, 0] / resolution, num_columns - 1)
        y_index = saturate(pcursor[:, :, 1] / resolution, num_rows - 1)
        
        # Now for the array of column and row indices, we want to
        # look up the sin and cos values
        x_step = step_length * cos_grid[x_index, y_index]
        y_step = step_length * sin_grid[x_index, y_index]

        # Increment the x, y offsets
        pcursor[:, :, 0] += x_step
        pcursor[:, :, 1] += y_step

    # for r in range(v_rows):
    #     for c in range(v_cols):
    #         # Render points
    #         frame = cv2.polylines(
    #             frame,
    #             [parr[r, c]],
    #             False,
    #             (255, 0, 0),
    #             thickness=2,
    #             lineType=cv2.LINE_AA,
    #         )

    frame = cv2.polylines(
        frame,
        parr.reshape((v_rows * v_cols, num_steps, 2)).astype(np.int32),
        False,
        (0, 255, 0),
        lineType=cv2.LINE_AA,
    )

    grid += np.pi * (0.2 / FPS)
    # lengths = (lengths + (1.0 / FPS)) % (10 * resolution)
    # frame = cv2.GaussianBlur(frame, (15, 15), 0)
    video.write(frame)

video.release()

In [ ]:
from IPython.display import Video

Video(FILENAME, embed=True)

In [ ]:
num_steps = 5
points = np.zeros((num_steps, 2), dtype=np.int32)
points[4][0] = 1.23
points